# B.1 - Ripeness Detection (2 Classes)

**Experiment:** B.1  
**Input:** RGB images (3-channel)  
**Objective:** End-to-end detection with ripeness classification  
**Classes:** 2 (ripe, unripe)

## Workflow
1. Load ripeness dataset (2-class YOLO format)
2. Train YOLOv11n with 5 seeds (42, 123, 456, 789, 101)
3. Evaluate on test set
4. Calculate mean ± std deviation

## Training Config
- Model: YOLOv11n
- Epochs: 100
- Patience: 30
- Other parameters: default

In [1]:
# =============================================================================
# Cell 1: Setup
# =============================================================================
import os
import torch
import numpy as np
import shutil
from pathlib import Path
from datetime import datetime

IS_KAGGLE = os.path.exists('/kaggle/input')
print(f"Running on: {'Kaggle' if IS_KAGGLE else 'Local'}")

# Paths
DATASET_PATH = Path('/kaggle/input/ffb-ripeness-detect-dataset/ffb_ripeness_detect')
BASE_PATH = Path('/kaggle/working')
RUNS_PATH = BASE_PATH / 'runs' / 'detect'
KAGGLE_OUTPUT = BASE_PATH / 'kaggleoutput'
KAGGLE_OUTPUT.mkdir(parents=True, exist_ok=True)

print(f"Dataset: {DATASET_PATH} (exists: {DATASET_PATH.exists()})")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Verify dataset
for split in ['train', 'val', 'test']:
    imgs = len(list((DATASET_PATH / 'images' / split).glob('*.png')))
    lbls = len(list((DATASET_PATH / 'labels' / split).glob('*.txt')))
    print(f"  {split}: {imgs} images, {lbls} labels")

Running on: Kaggle
Dataset: /kaggle/input/ffb-ripeness-detect-dataset/ffb_ripeness_detect (exists: True)
CUDA: True
GPU: Tesla T4
  train: 0 images, 280 labels
  val: 0 images, 80 labels
  test: 0 images, 40 labels


In [2]:
%%writefile /kaggle/working/dataset_ripeness.yaml
# B.1 Ripeness Detection Dataset (2 Classes)
path: /kaggle/input/ffb-ripeness-detect-dataset/ffb_ripeness_detect
train: images/train
val: images/val
test: images/test

nc: 2
names: ['ripe', 'unripe']

Writing /kaggle/working/dataset_ripeness.yaml


In [3]:
# =============================================================================
# Cell 3: Verify YAML
# =============================================================================
config_path = Path('/kaggle/working/dataset_ripeness.yaml')
print(f"YAML: {config_path}")
print(config_path.read_text())

YAML: /kaggle/working/dataset_ripeness.yaml
# B.1 Ripeness Detection Dataset (2 Classes)
path: /kaggle/input/ffb-ripeness-detect-dataset/ffb_ripeness_detect
train: images/train
val: images/val
test: images/test

nc: 2
names: ['ripe', 'unripe']



In [4]:
# =============================================================================
# Cell 4: Install Ultralytics
# =============================================================================
!pip install -q ultralytics

from ultralytics import YOLO
import pandas as pd
print("Ultralytics ready")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.8 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics ready


In [5]:
# =============================================================================
# Cell 5: Training Config
# =============================================================================
SEEDS = [42, 123, 456, 789, 101]
EXP_PREFIX = 'exp_b1_ripeness'

print(f"Seeds: {SEEDS} ({len(SEEDS)} runs)")
print("Metrics: mean ± std deviation")
print("Note: 2-class detection (ripe/unripe)")

Seeds: [42, 123, 456, 789, 101] (5 runs)
Metrics: mean ± std deviation
Note: 2-class detection (ripe/unripe)


In [6]:
# =============================================================================
# Cell 6: Training Loop
# =============================================================================
for seed in SEEDS:
    print(f"\n{'='*60}")
    print(f"TRAINING B.1 RIPENESS DETECTION - Seed {seed} ({SEEDS.index(seed)+1}/{len(SEEDS)})")
    print(f"{'='*60}\n")
    
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    
    model = YOLO('yolo11n.pt')
    
    results = model.train(
        data=str(config_path),
        epochs=100,
        patience=30,
        seed=seed,
        name=f"{EXP_PREFIX}_seed{seed}",
        exist_ok=True,
    )
    
    print(f"\nSeed {seed} complete!")
    print(f"mAP50: {results.results_dict.get('metrics/mAP50(B)', 0):.3f}")


TRAINING B.1 RIPENESS DETECTION - Seed 42 (1/5)

Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset_ripeness.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=exp_b1_ripeness_seed42, nbs=64, nms=False, opset=None, optimize=False, o

In [7]:
# =============================================================================
# Cell 7: Evaluation on Test Set
# =============================================================================
results_dict = {}

print("="*60)
print("EVALUATION ON TEST SET")
print("="*60)

for seed in SEEDS:
    model_path = RUNS_PATH / f"{EXP_PREFIX}_seed{seed}" / 'weights' / 'best.pt'
    
    if not model_path.exists():
        print(f"Not found: {model_path}")
        continue
    
    print(f"\nSeed {seed}:")
    model = YOLO(str(model_path))
    metrics = model.val(data=str(config_path), split='test')
    
    results_dict[seed] = {
        'mAP50': metrics.box.map50,
        'mAP50-95': metrics.box.map,
        'Precision': metrics.box.mp,
        'Recall': metrics.box.mr
    }
    
    print(f"  mAP50: {metrics.box.map50:.3f}")
    print(f"  mAP50-95: {metrics.box.map:.3f}")
    print(f"  Precision: {metrics.box.mp:.3f}")
    print(f"  Recall: {metrics.box.mr:.3f}")

EVALUATION ON TEST SET

Seed 42:
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 101 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 1.9±0.4 ms, read: 116.0±48.1 MB/s, size: 3110.6 KB)
val: Scanning /kaggle/input/ffb-ripeness-detect-dataset/ffb_ripeness_detect/labels/test... 40 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 40/40 177.0it/s 0.2s
WARNING ⚠️ val: Cache directory /kaggle/input/ffb-ripeness-detect-dataset/ffb_ripeness_detect/labels is not writable, cache not saved.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 1.1s/it 3.4s
                   all         40        140      0.861       0.78      0.846      0.554
                  ripe         17         22      0.809      0.773      0.787      0.496
                unripe         38        118      0.913      0.788      0.904      0.612
Speed: 2.8ms preprocess, 

In [8]:
# =============================================================================
# Cell 8: Results Summary
# =============================================================================
df = pd.DataFrame(results_dict).T
df.index.name = 'Seed'

# Calculate mean and std
avg = df.mean()
std = df.std()

print("\n" + "="*60)
print("B.1 RIPENESS DETECTION - FINAL RESULTS")
print("="*60 + "\n")
print(df.to_string(float_format=lambda x: f"{x:.3f}"))

print("\n" + "-"*60)
print("SUMMARY (Mean ± Std)")
print("-"*60)
for col in df.columns:
    print(f"  {col}: {avg[col]:.3f} ± {std[col]:.3f}")


B.1 RIPENESS DETECTION - FINAL RESULTS

      mAP50  mAP50-95  Precision  Recall
Seed                                    
42    0.846     0.554      0.861   0.780
123   0.805     0.538      0.718   0.806
456   0.807     0.549      0.797   0.712
789   0.811     0.504      0.784   0.734
101   0.800     0.518      0.732   0.799

------------------------------------------------------------
SUMMARY (Mean ± Std)
------------------------------------------------------------
  mAP50: 0.814 ± 0.018
  mAP50-95: 0.532 ± 0.021
  Precision: 0.779 ± 0.057
  Recall: 0.766 ± 0.042


In [9]:
# =============================================================================
# Cell 9: Save Results
# =============================================================================
output_file = KAGGLE_OUTPUT / 'b1_ripeness_results.txt'

with open(output_file, 'w') as f:
    f.write("="*60 + "\n")
    f.write("B.1 Ripeness Detection (2 Classes) Results\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("Training: epochs=100, patience=30, other=default\n")
    f.write(f"Seeds: {SEEDS}\n")
    f.write("="*60 + "\n\n")
    f.write("Per-Seed Results:\n")
    f.write(df.to_string(float_format=lambda x: f"{x:.3f}"))
    f.write("\n\n" + "-"*60 + "\n")
    f.write("Summary (Mean ± Std):\n")
    for col in df.columns:
        f.write(f"  {col}: {avg[col]:.3f} ± {std[col]:.3f}\n")

print(f"Results saved: {output_file}")

Results saved: /kaggle/working/kaggleoutput/b1_ripeness_results.txt


In [10]:
# =============================================================================
# Cell 10: Create Archives
# =============================================================================
if RUNS_PATH.exists():
    shutil.make_archive('/kaggle/working/b1_runs', 'zip', RUNS_PATH)
    print(f"b1_runs.zip: {os.path.getsize('/kaggle/working/b1_runs.zip')/1024/1024:.1f} MB")

shutil.make_archive('/kaggle/working/b1_output', 'zip', KAGGLE_OUTPUT)
print("b1_output.zip created")

print("\nDownload from Output tab")

b1_runs.zip: 126.9 MB
b1_output.zip created

Download from Output tab
